## 10-3 振動データの異常検知

### ライブラリのインポート

In [ ]:
import pandas as pd 
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(seed=0)

- sklearn.preprocessing.MinMaxScaler
  - Min-Max正規化の関数

### 独自関数の定義

In [ ]:
def CreateSubSequences(input_data, window_size, slide = 1):
    len_sequences = len(input_data)
    data = []
    from_idx, to_idx = 0, window_size
    while(to_idx < len_sequences):
        data.append(input_data[from_idx : to_idx])
        from_idx = from_idx + slide
        to_idx = to_idx + slide
    return(data)

- 入力されたデータを、窓幅に合わせて複数の部分列に分割し、Listとして返す関数
  - 引数1: Series型の時系列データ
  - 引数2: 窓幅
  - 引数3: 窓をスライドさせる幅（1~windowsize、デフォルト1）

### データの読み込み

In [ ]:
input_df =  pd.read_csv('data/qtdbsel102.csv', header=None)
input_df.head()

- pd.read_csv('data/qtdbsel102.csv', header=None)
  - CSVファイルのデータをPandasデータフレームへ読み込む。
  - header=None：ヘッダー行がない（1行目からデータである）ことを指定する。

### 分析対象データの描画

In [ ]:
ecg_data = input_df.loc[:, 2]
print('時系列データ長＝' + str(len(ecg_data)) )
plt.figure(figsize=(15, 5))
plt.plot(ecg_data)
plt.show()

- ecg_data = input_df.loc[:, 2]
  - 3列目のデータを取得する。

###  0〜5000の範囲を描画

In [ ]:
plt.figure(figsize=(15, 5))
plt.plot(ecg_data[0:5000])
plt.fill_between(range(4100, 4500), max(ecg_data), min(ecg_data), 
                 facecolor='orange', alpha=0.3)
plt.show()

- plt.fill_between( ･･･ )：指定範囲を塗りつぶす。
  - range(4100, 4500)：X軸の範囲を指定
  - max(ecg_data), min(ecg_data)：データの最大値と最小値の間を塗りつぶす。
  - facecolor：塗りつぶしの色
  - alpha：透明度の指定

### 訓練/テストデータの取得とハイパーパラメータの設定

In [ ]:
# 訓練/テストデータ
train_data = ecg_data[5000:45000]
test_data  = ecg_data[0:5000]
# ハイパーパラメータ
window_size = 400 # 窓幅（入力の次元）
hidden_unit = 64  # 中間層のユニット数
batch_size = 64   # バッチサイズ
epoch_n = 10      # エポック数

- train_data
  - 訓練データ。5000以降を正常データとして取得する。
- test_data
  - テストデータ。正常と異常が含まれている0〜5000のデータを使用する。

### データの準備

In [ ]:
# データ窓幅で区切って分割
train_data_list = CreateSubSequences(train_data, window_size)
test_data_list = CreateSubSequences(test_data, window_size)
# 正規化
scaler = MinMaxScaler()
train_data_list = scaler.fit_transform(train_data_list)
test_data_list  = scaler.transform(test_data_list)

- CreateSubSequences（独自関数）を使用して訓練データとテストデータを窓幅で区切って分割する。
- 分割後のデータを正規化する（最大値を1、最小値を0にする）。

### 学習モデルの構築

In [ ]:
model = Sequential()
model.add(Dense(hidden_unit, activation='relu', input_shape=(window_size,)))
model.add(Dense(window_size, activation='sigmoid'))

- Dense(hidden_unit, activation='relu', input_shape=(window_size,))
  - 入力層と中間層の定義
  - 入力数：window_size、ニューロン数：hidden_unit、活性化関数：relu
- Dense(window_size, activation='sigmoid')
  - 出力層の定義
  - 出力数：window_size、活性化関数：sigmoid 

### 学習の実行

In [ ]:
model.compile(loss='mean_squared_error', optimizer ='adam')
history = model.fit(train_data_list,train_data_list,
                    epochs=epoch_n,batch_size=batch_size, shuffle=True)

- model.compile( ･･･ )
  - 損失関数：MSE（平均二乗誤差）、学習アルゴリズム：Adam
- model.fit( ･･･ )
  - shuffle=True：学習の偏りを減らすため、訓練データを各エポック前にシャッフルする。

### 損失関数出力のプロット

In [ ]:
pd.DataFrame(history.history['loss'], columns=['loss']).plot()
plt.show()

- 損失関数の変化を見ると学習の収束度合いが判断できる。

### 学習済みモデルの評価

In [ ]:
decoded = model.predict(test_data_list)
decoded_df = pd.DataFrame(decoded)
# 異常度スコアの計算（入力されたテストデータとモデルで復元したデータとの差分）
anomaly_score = np.sqrt( np.sum((decoded - test_data_list)**2, axis=1))

- テストデータと復元結果で差分が生じる。
  - 訓練データに含まれていない波形が入力されると、復元がうまくいかず差分が大きくなる。
- 差分を「Anomaly Score（異常度スコア）」として異常を検知する。
  - 異常度スコアは相対的な値である。
  - 絶対値で閾値を設けるのでなく他の箇所との比較で判断する。

### 異常度スコアの描画

In [ ]:
plt.figure(figsize=(15,5))
plt.subplot(2,1,1)
plt.title('Anomaly Score')
plt.plot(anomaly_score)
plt.subplot(2,1,2)
plt.title('Test Data')
plt.plot(test_data)
plt.show()

### 最も異常度スコアが低い波形のピックアップ

In [ ]:
plt.figure(figsize=(10,5))
anomaly_idx = np.where(anomaly_score == min(anomaly_score))
plt.title('Minimum anomaly score window is ' + str(int(anomaly_idx[0])))
plt.plot(test_data_list[anomaly_idx].T , label='Test')
plt.plot(decoded[anomaly_idx].T , label='Predict')
plt.legend()
plt.show()

### 最も異常度スコアが高い波形のピックアップ

In [ ]:
plt.figure(figsize=(10,5))
anomaly_idx = np.where(anomaly_score == max(anomaly_score))
plt.title('Most anomaly window is ' + str(int(anomaly_idx[0])))
plt.plot(test_data_list[anomaly_idx].T , label='Test')
plt.plot(decoded[anomaly_idx].T , label='Predict')
plt.legend()
plt.show()